In [1]:
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '..')

# for saving
import os

from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

from src import setup
# setup.use_gpu()

In [2]:
from src import ( 
    measurements as measure,
    density_matrix as DM,
    simulation as sim,
    orders,
    random_unitary,
    simulation)

In [3]:
identity = DM.Identity(DM.energy_basis(4))
print(identity.data.blocks)

[array([[1.+0.j]]), array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]], dtype=complex64), array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]], dtype=complex64), array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]], dtype=complex64), array([[1.+0.j]])]


In [4]:
num_iterations = 200
measurments = [measure.extractable_work_of_each_qubit]
num_samples = 50

In [8]:
titles = ["six","five","messenger"]
results_overall = {8:defaultdict(lambda: defaultdict(list)),12:defaultdict(lambda: defaultdict(list)),16:defaultdict(lambda: defaultdict(list))}

for num_qbits in [8]:
    print(f"working on {num_qbits}")
    # orderings
    # gas_orderings = orders.n_random_gas_orders(num_qbits,chunk_sizes = [4 for _ in range(num_qbits//4)],n = num_iterations)
    line_orderings= orders.n_random_line_orders(num_qbits,chunk_sizes = [4 for _ in range(num_qbits//4)],n = num_iterations)
    five_orderings = orders.n_alternating_c5_orders(num_qbits,n = num_iterations)
    messenger_orderings = orders.n_alternating_messenger_orders(num_qbits,n=num_iterations)

    orderings = [line_orderings,five_orderings,messenger_orderings]
    
    initial_pops = [.2 for _ in range(num_qbits)]
    initial_pops[0] = .4

    for i,ordering in enumerate(orderings):
        print(f"working on connectivity {titles[i]}")
                                                     


        results = results_overall[num_qbits]

        results[titles[i]]["pops"] = []
        results[titles[i]]["ex_work"] = []

        for index in range(num_samples):

            sub_unitary = random_unitary.random_unitary_in_subspace(4,2)

            unitary_list = []
            for unitary_index in range(num_qbits):
                piece = DM.tensor([sub_unitary if j==unitary_index else identity for j in range(num_qbits//4)])
                unitary_list.append(piece)
            unitary = np.product(unitary_list)

            system = DM.n_thermal_qbits(initial_pops)
            print(system.basis == )

            data = sim.run(system,
                              measurement_set = measurments,
                              num_iterations = num_iterations,
                              orders = ordering,
                              Unitaries = unitary
                              )[0];

            if index %1==0: print(index)

            # results[titles[i]]["pops"].append(data[0])
            results[titles[i]]["ex_work"].append(data[0])

working on 8
working on connectivity six
[0 1 2 4 8 16 32 64 128 3 5 6 9 10 12 17 18 20 24 33 34 36 40 48 65 66 68 72 80 96 129 130 132 136 144 160 192 7 11 13 14 19 21 22 25 26 28 35 37 38 41 42 44 49 50 52 56 67 69 70 73 74 76 81 82 84 88 97 98 100 104 112 131 133 134 137 138 140 145 146 148 152 161 162 164 168 176 193 194 196 200 208 224 15 23 27 29 30 39 43 45 46 51 53 54 57 58 60 71 75 77 78 83 85 86 89 90 92 99 101 102 105 106 108 113 114 116 120 135 139 141 142 147 149 150 153 154 156 163 165 166 169 170 172 177 178 180 184 195 197 198 201 202 204 209 210 212 216 225 226 228 232 240 31 47 55 59 61 62 79 87 91 93 94 103 107 109 110 115 117 118 121 122 124 143 151 155 157 158 167 171 173 174 179 181 182 185 186 188 199 203 205 206 211 213 214 217 218 220 227 229 230 233 234 236 241 242 244 248 63 95 111 119 123 125 126 159 175 183 187 189 190 207 215 219 221 222 231 235 237 238 243 245 246 249 250 252 127 191 223 239 247 251 253 254 255]


AssertionError: 